# Circle

In [20]:
%%capture
!pip install git+https://github.com/fuyu-quant/IBLM.git@feature-in-context-learning 

In [21]:
#%%capture
#!pip install git+https://github.com/fuyu-quant/IBLM.git

In [22]:
import pkg_resources
print(pkg_resources.get_distribution('IBLM').version)

0.1.44


In [23]:
import pandas as pd
import numpy as np
from langchain.llms import OpenAI
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from iblm import IBLMClassifier

import sys
import os

In [24]:
n = 400

df = pd.read_csv(f'../data/circle/circle_{n}_train.csv')
x_train = df.drop('Target', axis=1)
y_train = df['Target']
print(len(x_train))

400


In [25]:
llm_model = OpenAI(temperature=0, model_name = 'gpt-4')

params = {'columns_name': True}

iblm = IBLMClassifier(llm_model = llm_model, params=params)

### Training

In [39]:
file_path = '../models/circle/'

model = iblm.fit(x_train, y_train, model_name = 'circle', file_path=file_path)

Tokens Used: 5557
	Prompt Tokens: 5355
	Completion Tokens: 202
Successful Requests: 1
Total Cost (USD): $0.17277


In [40]:
# Code of the model created
print(model)

import numpy as np

def predict(x):
    df = x.copy()
    output = []
    for index, row in df.iterrows():
        # Do not change the code before this point.
        # Please describe the process required to make the prediction below.

        feature_1 = row['Feature_1']
        feature_2 = row['Feature_2']

        # Calculate the distance from the origin (0, 0)
        distance = np.sqrt(feature_1**2 + feature_2**2)

        # Normalize the distance to a probability value between 0 and 1
        probability = 1 / (1 + np.exp(-distance))

        # Threshold for classification
        threshold = 0.5

        # Assign the probability value to y
        if probability > threshold:
            y = 1
        else:
            y = 0

        # Do not change the code after this point.
        output.append(y)
    return np.array(output)


### Prediction

In [41]:
df = pd.read_csv(f'../data/circle/circle_{n}_test.csv')
x_test = df.drop('Target', axis=1)
y_test = df['Target']

In [42]:
y_proba = iblm.predict(x_test)
y_pred = (y_proba > 0.5).astype(int)

In [43]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {roc_auc}')

Accuracy: 0.5
Precision: 0.5
Recall: 1.0
F1 score: 0.6666666666666666
ROC-AUC: 0.5


### Create multiple code models

In [44]:
n = 600
df = pd.read_csv(f'../data/circle/circle_{n}_train.csv')
x_train = df.drop('Target', axis=1)
y_train = df['Target']

file_path = '../models/circle/'
for i in range(1,31):
    model = iblm.fit(x_train, y_train, model_name = f'circle_{i}', file_path=file_path)

Tokens Used: 4835
	Prompt Tokens: 4633
	Completion Tokens: 202
Successful Requests: 1
Total Cost (USD): $0.15110999999999997
Tokens Used: 4835
	Prompt Tokens: 4633
	Completion Tokens: 202
Successful Requests: 1
Total Cost (USD): $0.15110999999999997
Tokens Used: 4830
	Prompt Tokens: 4633
	Completion Tokens: 197
Successful Requests: 1
Total Cost (USD): $0.15080999999999997
Tokens Used: 4830
	Prompt Tokens: 4633
	Completion Tokens: 197
Successful Requests: 1
Total Cost (USD): $0.15080999999999997
Tokens Used: 4835
	Prompt Tokens: 4633
	Completion Tokens: 202
Successful Requests: 1
Total Cost (USD): $0.15110999999999997
Tokens Used: 4830
	Prompt Tokens: 4633
	Completion Tokens: 197
Successful Requests: 1
Total Cost (USD): $0.15080999999999997
Tokens Used: 4830
	Prompt Tokens: 4633
	Completion Tokens: 197
Successful Requests: 1
Total Cost (USD): $0.15080999999999997
Tokens Used: 4835
	Prompt Tokens: 4633
	Completion Tokens: 202
Successful Requests: 1
Total Cost (USD): $0.15110999999999997


### Prediction

In [46]:
n = 400

df = pd.read_csv(f'../data/circle/circle_{n}_test.csv')
x_test = df.drop('Target', axis=1)
y_test = df['Target']

sys.path.append('..')

error_count = 0
error_list = []
auc_list = []


for i in range(1,31):

    import_file = f'import models.circle.circle_{i} as codemodel'

    exec(import_file)

    try:
        y_proba = codemodel.predict(x_test)
        y_pred = (y_proba > 0.5).astype(int)
        negative_values_exist = np.any(y_proba < 0)
        values_greater_than_one_exist = np.any(y_proba > 1)
        if negative_values_exist:
            error_list.append(i)
            error_count += 1
            print(f"Negative values exist：{negative_values_exist}")

        elif values_greater_than_one_exist:
            error_list.append(i)
            error_count += 1
            print(f"Positive values exist：{values_greater_than_one_exist}")

        else:
            roc_auc = roc_auc_score(y_test, y_proba)
            auc_list.append(roc_auc)
        
    except Exception:
        print('run error')
        error_count += 1
        error_list.append(i)
        pass

print(error_count)
print(error_list)
print(auc_list)
average = sum(auc_list) / len(auc_list)
print("Average Value:", average)
max_value = max(auc_list)
min_value = min(auc_list)
print("Maximum Value:", max_value)
print("Minimum Value:", min_value)

0
[]
[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.3919777777777778, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.9953333333333334, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
Average Value: 0.5129103703703703
Maximum Value: 0.9953333333333334
Minimum Value: 0.3919777777777778


In [48]:
n = 400
df = pd.read_csv(f'../data/circle/circle_{n}_test.csv')
x_train = df.drop('Target', axis=1)
y_train = df['Target']

import models.circle.circle_17 as codemodel

y_proba = codemodel.predict(x_test)
y_pred = (y_proba > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {roc_auc}')

Accuracy: 0.5166666666666667
Precision: 1.0
Recall: 0.03333333333333333
F1 score: 0.06451612903225806
ROC-AUC: 0.9953333333333334
